In [17]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

In [19]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\train', transform=transform)
test_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\test', transform=transform)
val_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)


In [20]:
conv1 = nn.Conv2d(3, 32, 3)
conv2 = nn.Conv2d(32, 64, 3)
conv3 = nn.Conv2d(64, 128, 3)
pool = nn.MaxPool2d(2)
fc1 = nn.Linear(128 * 26 * 26, 256)
fc2 = nn.Linear(256, 128)
fc3 = nn.Linear(128, 1)
dropout = nn.Dropout(0.3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv1.to(device)
conv2.to(device)
conv3.to(device)
fc1.to(device)
fc2.to(device)
fc3.to(device)
pool.to(device)
dropout.to(device)


def forward(x):
    x = pool(F.leaky_relu(conv1(x)))
    x = dropout(x)
    x = pool(F.leaky_relu(conv2(x)))
    x = dropout(x)
    x = pool(F.leaky_relu(conv3(x)))
    x = dropout(x)
    x = x.view(-1, 128 * 26 * 26)
    x = F.leaky_relu(fc1(x))
    x = dropout(x)
    x = F.leaky_relu(fc2(x))
    x = dropout(x)
    x = torch.sigmoid(fc3(x))
    return x
criterion = nn.BCELoss()
optimizer = optim.Adam(list(conv1.parameters()) + list(conv2.parameters()) + list(conv3.parameters()) + 
                       list(fc1.parameters()) + list(fc2.parameters()) + list(fc3.parameters()), lr=0.001)

In [21]:
for epoch in range(20):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = forward(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch {epoch+1}, loss: {running_loss / len(train_loader)}")


epoch 1, loss: 0.688864739496232
epoch 2, loss: 0.4587880041045967
epoch 3, loss: 0.1993015060619683
epoch 4, loss: 0.09929854154330497
epoch 5, loss: 0.04200903795814762
epoch 6, loss: 0.05170277208395249
epoch 7, loss: 0.034726720715893895
epoch 8, loss: 0.03988544266233708
epoch 9, loss: 0.026251565403501355
epoch 10, loss: 0.02059061631780071
epoch 11, loss: 0.021156554580380727
epoch 12, loss: 0.01952823551762903
epoch 13, loss: 0.007995286864748739
epoch 14, loss: 0.009291155921994849
epoch 15, loss: 0.013742774269610153
epoch 16, loss: 0.016588246367570114
epoch 17, loss: 0.016303174611035615
epoch 18, loss: 0.014402987953336244
epoch 19, loss: 0.011176088380018436
epoch 20, loss: 0.020819726253366682


In [22]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print(f"test accuracy: {100 * correct / total}%")

test accuracy: 98.24561403508773%


In [23]:
torch.save({
    'conv1_state_dict': conv1.state_dict(),
    'conv2_state_dict': conv2.state_dict(),
    'conv3_state_dict': conv3.state_dict(),
    'fc1_state_dict': fc1.state_dict(),
    'fc2_state_dict': fc2.state_dict(),
    'fc3_state_dict': fc3.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'main3.pth')

In [24]:
from sklearn.metrics import precision_recall_fscore_support
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

print(f"Test Accuracy: {100 * correct / total}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Test Accuracy: 98.24561403508773%
Precision: 0.961352657004831
Recall: 0.995
F1 Score: 0.9778869778869779
